In [1]:
import numpy as np
from scipy.stats import unitary_group
from functools import reduce
np.set_printoptions(linewidth=130)

U1 = unitary_group.rvs(4).reshape(2,2,2,2);
U2 = unitary_group.rvs(4).reshape(2,2,2,2);
U3 = unitary_group.rvs(4).reshape(2,2,2,2);

U1_ = U1.reshape(4,4).conj().T.reshape(2,2,2,2)
U2_ = U2.reshape(4,4).conj().T.reshape(2,2,2,2)
U3_ = U3.reshape(4,4).conj().T.reshape(2,2,2,2)

T1 = lambda U1, U2, U3: np.einsum(
    U1, [12,13,8,9],
    U1, [17,19,10,11],
    U2, [14,15,7,12],
    U2, [16,18,13,17],
    U3, [0,1,6,14],
    U3, [2,3,15,16],
    U3, [4,5,18,19],

    [0,1,2,3,4,5,6,7,8,9,10,11]
)

T2 = lambda U1_, U2_, U3_: np.einsum(
    U3_, [0,12,6,7],
    U3_, [13,14,8,9],
    U3_, [18,19,10,11],
    U2_, [1, 16,12,13],
    U2_, [17,15,14,18],
    U1_, [2,3,16,17],
    U1_, [4,5,15,19],
    [0,1,2,3,4,5,6,7,8,9,10,11]
)

###########################################################
# Test that T1 and T2 are inverses of eachother
###########################################################


I = np.einsum(
    T1(U1,U2,U3), [0,1,2,3,4,5,12,13,14,15,16,17],
    T2(U1_,U2_,U3_), [12,13,14,15,16,17,6,7,8,9,10,11],
    [0,1,2,3,4,5,6,7,8,9,10,11]
)

assert np.allclose(T1(U1,U2,U3).reshape(64,64) @ T2(U1_,U2_,U3_).reshape(64,64), np.eye(64))
assert np.allclose(I.reshape(64,64), np.eye(64))
############################################################
#
###########################################################
T3 = lambda T1, T2: np.einsum(
    T1, [0,1,2,3,4,5,12,13,16,17,18,19],
    T2, [14,15,16,17,18,19,6,7,8,9,10,11],
    [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
)

T4 = lambda U1,U2,U3,U1_,U2_,U3_: np.einsum(
    U3_, [10,12,4,5],
    U2_, [11,13,12,6],
    U1_, [14,15,13,7],
    U1,  [16,3,14,15],
    U2,  [17,2,9,16],
    U3,  [0,1,8,17],
    [0,1,2,3,4,5,6,7,8,9,10,11]
)
##################################
# test that T4 works as expcted
##################################


assert np.allclose(T4(np.eye(4).reshape(2,2,2,2), U2, U3, np.eye(4).reshape(2,2,2,2), U2_, U3_)[0,0,0,0,0,0,0,0,...].reshape(4,4), T4(U1, U2, U3, U1_, U2_, U3_)[0,0,0,0,0,0,0,0,...].reshape(4,4))
#####################################
#
#####################################

T5 = lambda T3, T4: np.einsum(
    T3, [2,3,4,5,6,7,10,11,12,13,14,15,21,20,23,22],
    T4, [0,1,21,20,8,9,23,22,16,17,18,19],
    [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
)

T5R = lambda T5: np.round(T5[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...].reshape(4,4),4)
T3R = lambda T3: np.round(T3[0,0,0,0,0,0,0,0,0,0,0,0,...].reshape(4,4),4)


U = [U1,U2,U3,U1_,U2_,U3_]

t1 = T1(*U[:3])
t2 = T2(*U[3:])
t3 = T3(t1,t2)
t4 = T4(*U)
t5 = T5(t3, t4)
t5r = T5R(t5)
t3r = T3R(t3)

assert(np.allclose(t5r, t3r))

def rfp(U): #right fixed point
    t1 = T1(*U[:3])
    t2 = T2(*U[3:])
    t3 = T3(t1,t2)
    t4 = T4(*U)
    t5 = T5(t3, t4)
    t5r = T5R(t5)
    t3r = T3R(t3)
    return t5r, t3r

t5r1, t3r1 = rfp(U)

assert np.allclose(t5r1, t3r1)



In [2]:
from scipy.linalg import expm

U1 = unitary_group.rvs(4).reshape(2,2,2,2)
U2 = unitary_group.rvs(4).reshape(2,2,2,2)
U3 = unitary_group.rvs(4).reshape(2,2,2,2)

M1 = np.random.rand(4,4) + 1j * np.random.rand(4,4)
B1 = 0.5 * (M1 - M1.conj().T)
M2 = np.random.rand(4,4) + 1j * np.random.rand(4,4)
B2 = 0.5 * (M2 - M2.conj().T)
M3 = np.random.rand(4,4) + 1j * np.random.rand(4,4)
B3 = 0.5 * (M3 - M3.conj().T)

U1_ = (expm(0.3 * B1)@ U1.reshape(4,4)).reshape(2,2,2,2)
U2_ = (expm(0.3 * B2)@ U2.reshape(4,4)).reshape(2,2,2,2)
U3_ = (expm(0.3 * B3)@ U3.reshape(4,4)).reshape(2,2,2,2)

U = [U1,U2,U3,U1_,U2_,U3_]

a, b = rfp(U)

In [3]:
print(a)

[[-0.0022+0.006j   0.0136-0.0126j -0.008 -0.012j   0.0404+0.0262j]
 [ 0.0019+0.0059j  0.0091-0.0124j -0.0049-0.004j   0.0309+0.0212j]
 [ 0.0072-0.0254j -0.0384+0.0333j  0.0258+0.0288j -0.0923-0.0931j]
 [ 0.0044-0.0071j -0.0157+0.0088j -0.0023+0.0124j -0.0174-0.0528j]]


In [4]:
print(b)

[[ 0.0008-2.300e-03j -0.0091-3.050e-02j -0.0382+3.400e-03j  0.0525-6.370e-02j]
 [ 0.0016-1.000e-04j -0.0004-2.320e-02j -0.0144-3.300e-03j  0.0527-1.520e-02j]
 [-0.0127-3.610e-02j  0.0229+1.064e-01j  0.0962-2.130e-02j -0.2067+8.590e-02j]
 [-0.0047-1.020e-02j -0.0079+3.020e-02j  0.0219+1.210e-02j -0.0911-2.010e-02j]]


In [6]:
a * (b[0,0]/a[0,0])

array([[ 0.0008    -0.0023j    , -0.00510137+0.00488717j,  0.00312439+0.00452106j, -0.01555916-0.00972498j],
       [-0.00076146-0.0022358j , -0.00338815+0.00478232j,  0.00189236+0.0014928j , -0.01190784-0.00788046j],
       [-0.00258149+0.00972321j,  0.01441836-0.01293173j, -0.01001312-0.01080852j,  0.03575891+0.0348834j ],
       [-0.00163119+0.0027331j ,  0.00592566-0.00345274j,  0.00079736-0.00473903j,  0.00696552+0.02000597j]])

In [8]:
print(a / b)

[[-2.62394604-0.04384486j  0.25718121+0.52263439j  0.18003807+0.33016046j  0.06634341+0.57954429j]
 [ 0.95330739+3.74708171j  0.52756315+0.4013373j   0.38378007+0.18982818j  0.43419106+0.52750862j]
 [ 0.56367361+0.39774667j  0.22487774+0.40930169j  0.19246961+0.34199171j  0.22116371+0.54232202j]
 [ 0.41021169+0.62039166j  0.40001026+0.4152291j   0.15921217+0.47824351j  0.30407366+0.51249308j]]
